In [1]:
pip install elapid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 824.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.8 MB/s eta 0:00:00


In [2]:
pip install geopandas

In [3]:
import geopandas as gpd
import elapid as ela
from sklearn import metrics # For Evaluation Metrics (To assess the performance)


import geopandas as gpd #for Geospatial Data, specifically Vector Data and other formats such as shapefiles, GeoJSON & Geopackage files
import rasterio as rio # Reading, Writing, and Manipulating Raster Geospatial Data
import rasterio.plot as rioplot # To Visualize Raster Data
from sklearn import metrics # For Evaluation Metrics (To assess the performance)
import matplotlib as mpl # For Data Visualization
import matplotlib.pyplot as plt # For Data Visualization
from rasterio.plot import show # To Display the raster data using the show function

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
bioclimate_variables = '/content/drive/MyDrive/Project_2/Insect Pest/tif files/2022 crop Argiculture data for US/2022_30m_cdls/2022_30m_cdls.tif'

In [6]:
vector = '/content/drive/MyDrive/Project_2/Insect Pest/Species Data/EDD Maps site/Halyomorpha_halys(1999_2023).gpkg'

In [7]:

# Read the presence data, draw background point samples
presence = gpd.read_file(vector)
print(presence.crs)  # Verify the CRS of the presence geodataframe

# Check for empty Points and filter them out
presence = presence[~presence.geometry.is_empty]

background1 = ela.sample_raster(bioclimate_variables, count=10_000)
print(background1.crs)  # Verify the CRS of the background1 geodataframe

# Specify CRS directly
presence.crs = background1.crs
print(presence.crs)  # Verify the updated CRS of the presence geodataframe

# Merge datasets and read the covariates at each point location
merged = ela.stack_geodataframes(presence, background1, add_class_label=True)
print(merged.head())

annotated = ela.annotate(merged, bioclimate_variables, drop_na=True, quiet=True)
print(annotated.head())


EPSG:4326
PROJCS["NAD83 / Conus Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","5070"]]
PROJCS["NAD83 / Conus Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic

In [8]:
# split the x/y data
x = annotated.drop(columns=['class', 'geometry'])
y = annotated['class']

In [9]:
# train the model
model = ela.MaxentModel(transform='cloglog', beta_multiplier=2.0)
model.fit(x, y)

MaxentModel(beta_multiplier=2.0)

In [10]:
# evaluate training performance
ypred = model.predict(x)
auc = metrics.roc_auc_score(y, ypred)
print(f"Training AUC score: {auc:0.3f}")


Training AUC score: 0.789


In [11]:
import numpy as np
import matplotlib.pyplot as plt

# Example values
height = 3
width = 2

# Adjust the dimensions if necessary
if height * width != len(ypred):
    width = len(ypred) // height
    if height * width != len(ypred):
        print("Length of ypred:", len(ypred))
        print("Calculated height:", height)
        print("Calculated width:", width)
        raise ValueError("Cannot reshape array to the specified dimensions.")

# Reshape the array with the adjusted dimensions
ypred_2d = np.reshape(ypred, (height, width))

# Plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
ax.imshow(ypred_2d, cmap='hot', interpolation='nearest')
plt.show()


Length of ypred: 23797
Calculated height: 3
Calculated width: 7932


ValueError: ignored

In [ ]:
# Example values
height = 3
width = 2

# Adjust the dimensions if necessary
if height * width != len(ypred):
    width = len(ypred) // height
    if height * width != len(ypred):
        raise ValueError("Cannot reshape array to the specified dimensions.")

# Reshape the array with the adjusted dimensions
ypred_2d = np.reshape(ypred, (height, width))

# Plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
ax.imshow(ypred_2d, cmap='hot', interpolation='nearest')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Determine the dimensions based on the size of ypred
size = ypred.size
height = int(np.sqrt(size))
width = (size // height) + 1

# Calculate the new size based on the dimensions
new_size = height * width

# Adjust the dimensions if the new size doesn't match the original size
if new_size > size:
    width -= 1

# Reshape the array with the adjusted dimensions
ypred_2d = np.reshape(ypred, (height, width))

# Plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(ypred_2d, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Brown Marmorated Stink Bug suitability')
ax.set_xticks([])
ax.set_yticks([])

plt.show()


In [ ]:
# Determine the dimensions based on the size of ypred
size = ypred.size
height = int(np.sqrt(size))
width = size // height

# Reshape the array with the determined dimensions
ypred_2d = np.reshape(ypred, (height, width))

# Plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
plot = ax.imshow(ypred_2d, vmin=0, vmax=1, cmap='YlGnBu_r')
ax.set_title('Brown Marmorated Stink Bug suitability')
ax.set_xticks([])
ax.set_yticks([])

plt.show()


In [ ]:
# Replace 'ypred' with your actual array variable

# Determine the dimensions of the reshaped array based on the actual size of 'ypred'
size = ypred.size
height = int(np.sqrt(size))  # Example: Assume the height is the square root of the array size
width = size // height

# Reshape the array with the determined dimensions
ypred_2d = np.reshape(ypred, (height, width))

# Plot the suitability predictions
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=100)
ax.imshow(ypred_2d, cmap='viridis')
plt.show()


In [ ]:
selected_data=np.where((data>=min_temperature)&(data<=max_temperature), data, np.nan)
mask_temperature=np.where((data>=min_temperature)&(data<=max_temperature), data, np.nan)
mask_precipitation =np.where((data>=min_temperature)&(data<=max_temperature), data, np.nan)